In [ ]:
def generate_structural_diagram_with_slabs(obj_file, grid_size=6, floor_height=4, num_floors=3):
    """
    Generates a structural diagram with:
    - Core walls (forming a closed box)
    - Optimized column placement (around cores and perimeter)
    - Beams forming a logical structural grid
    - Slabs at each floor level
    """
    # Load the 3D model
    mesh = trimesh.load_mesh(obj_file)

    # Compute bounding box for grid & structure
    bounds = mesh.bounds
    min_x, min_y, min_z = bounds[0]
    max_x, max_y, max_z = bounds[1]

    # Detect Core Area: Bounding Box of Lower 20% of Mass
    core_z_threshold = (max_z - min_z) * 0.2
    core_mask = mesh.vertices[:, 2] < min_z + core_z_threshold
    core_points = mesh.vertices[core_mask]

    # Create a Tight Bounding Box Around Core Area
    if len(core_points) > 3:
        core_min_x, core_min_y = np.min(core_points[:, :2], axis=0)
        core_max_x, core_max_y = np.max(core_points[:, :2], axis=0)
        core_corners = [
            (core_min_x, core_min_y, min_z), (core_max_x, core_min_y, min_z),
            (core_max_x, core_max_y, min_z), (core_min_x, core_max_y, min_z)
        ]
        core_walls = [(core_corners[i], core_corners[(i + 1) % 4]) for i in range(4)]
    else:
        core_walls = []

    # Generate Column Grid: Place at Core Corners & Perimeter
    x_positions = np.arange(min_x, max_x + 1, grid_size)
    y_positions = np.arange(min_y, max_y + 1, grid_size)
    columns = [(x, y, min_z) for x in x_positions for y in y_positions
               if (x <= core_min_x or x >= core_max_x) or (y <= core_min_y or y >= core_max_y)]

    # Extend Columns Vertically (Multi-Floor)
    full_columns = [(x, y, z) for x, y, _ in columns for z in np.arange(min_z, max_z, floor_height)]

    # Define Beams Between Columns (Structurally Logical Connections)
    beams = []
    for col in columns:
        x, y, z = col
        neighbors = [(x + grid_size, y, z), (x, y + grid_size, z)]
        for n in neighbors:
            if n in columns:
                beams.append((col, n))

    # Define Slabs (Horizontal Planes at Each Floor Level)
    slabs = []
    for floor in range(1, num_floors + 1):
        slab_z = min_z + floor * floor_height
        slab_corners = [
            (min_x, min_y, slab_z), (max_x, min_y, slab_z),
            (max_x, max_y, slab_z), (min_x, max_y, slab_z)
        ]
        slabs.append(slab_corners)

    # 📌 Structural Visualization in 3D
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Plot Core Walls (Closed Box)
    for wall in core_walls:
        p1, p2 = wall
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [min_z, min_z + max_z * 0.6],
                color='orange', lw=3, label='Core Walls' if wall == core_walls[0] else "")

    # Plot Columns (Optimized)
    for col in full_columns:
        ax.scatter(*col, color='r', s=50, label='Columns' if col == full_columns[0] else "")

    # Plot Beams (Connecting Structural Grid)
    for beam in beams:
        p1, p2 = beam
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], color='b', lw=1.5,
                label='Beams' if beam == beams[0] else "")

    # Plot Slabs (Horizontal Layers)
    for slab in slabs:
        x_vals = [p[0] for p in slab] + [slab[0][0]]
        y_vals = [p[1] for p in slab] + [slab[0][1]]
        z_vals = [p[2] for p in slab] + [slab[0][2]]
        ax.plot(x_vals, y_vals, z_vals, color='purple', lw=2, linestyle='dotted', label='Slabs' if slab == slabs[0] else "")

    # Labels & Display
    ax.set_xlabel("X Axis (m)")
    ax.set_ylabel("Y Axis (m)")
    ax.set_zlabel("Height (m)")
    plt.legend()
    plt.title("Optimized Structural Diagram with Slabs & Improved Column Placement")
    plt.show()

# Run the improved structural visualization
generate_structural_diagram_with_slabs("/mnt/data/sample_model.obj")